In [1]:
import pyterrier  as pt
from tqdm import tqdm
import pandas as pd
import time

In [2]:
pt.init()

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
indexref = pt.IndexRef.of("../syn-question-col-analysis/datasets/msmarco/terrier_index/")
index = pt.IndexFactory.of(indexref)



13:41:07.070 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.


In [4]:


token2id = {word.getKey():i for i,word in enumerate(index.getLexicon()) }


In [5]:
def tp_func():
  stops = pt.autoclass("org.terrier.terms.Stopwords")(None)
  stemmer = pt.autoclass("org.terrier.terms.PorterStemmer")(None)
  def _apply_func(row):
    words = row["query"].split(" ") # this is safe following pt.rewrite.tokenise()
    words = [stemmer.stem(w) for w in words if not stops.isStopword(w) ]
    return " ".join(words)
  return _apply_func 

pipe = pt.rewrite.tokenise() >> pt.apply.query(tp_func())

In [6]:
import pandas as pd

pipe(pd.DataFrame([{"qid":0, "query":"hello my name is studing".lower()}]))["query"][0]

'hello name stude'

In [7]:
a = pd.DataFrame([{"qid":0, "query":"hello my name is studing".lower()}])#[0,0]

In [8]:
import json

with open("../syn-question-col-analysis/question_generation/gen_output/msmarco/selected_corpus_lm_fcm_STD2_L10000_gpt-neo-1.3B_BS_5_E13931.459746599197.jsonl") as f:
    questions = [line for line in map(json.loads, f)]

In [10]:
for at in [10, 100, 1000, 10_000]:
    bm25_pipe = pt.rewrite.tokenise() >> (pt.BatchRetrieve(index, wmodel="BM25", num_results=at))
    results = []
    time_list = []
    st = time.time()
    for i in tqdm(range(200)):
        question = questions[i]
        
        questions_dataframe = pd.DataFrame([{"qid":0, "query":question["question"].lower()}])

        df_results = bm25_pipe.transform(questions_dataframe)
        
        results.append(df_results["docno"].tolist())
    print(at, ":", 200/(time.time()-st),"it/s")

100%|██████████| 200/200 [00:09<00:00, 20.80it/s]


10 : 20.795093903445817 it/s


100%|██████████| 200/200 [00:10<00:00, 18.38it/s]


100 : 18.3788653809974 it/s


100%|██████████| 200/200 [00:22<00:00,  8.87it/s]


1000 : 8.872609842918274 it/s


100%|██████████| 200/200 [02:36<00:00,  1.27it/s]

10000 : 1.2742824856603254 it/s


In [13]:
sum(time_list)/len(time_list)

0.055793352603912355

In [9]:
bm25_pipe = pt.rewrite.tokenise() >> (pt.BatchRetrieve(index, wmodel="BM25", num_results=10, controls={"bm25.k3" : 0}))

question = questions[0]

questions_dataframe = pd.DataFrame([{"qid":0, "query":question["question"].lower()}])

df_results = bm25_pipe.transform(questions_dataframe)

In [10]:
df_results

,qid,docid,docno,rank,score,query_0,query
0,0,341,341,0,73.502843,what is the relationship between rarotongan an...,what is the relationship between rarotongan an...
1,0,6080047,6080047,1,43.770319,what is the relationship between rarotongan an...,what is the relationship between rarotongan an...
2,0,6080050,6080050,2,43.179743,what is the relationship between rarotongan an...,what is the relationship between rarotongan an...
3,0,6080052,6080052,3,41.241986,what is the relationship between rarotongan an...,what is the relationship between rarotongan an...
4,0,2787655,2787655,4,40.527819,what is the relationship between rarotongan an...,what is the relationship between rarotongan an...
5,0,3224336,3224336,5,40.233040,what is the relationship between rarotongan an...,what is the relationship between rarotongan an...
6,0,4383297,4383297,6,38.802598,what is the relationship between rarotongan an...,what is the relationship between rarotongan an...
7,0,5254867,5254867,7,38.565208,what is the relationship between rarotongan an...,what is the relationship between rarotongan an...
8,0,3005847,3005847,8,37.762542,what is the relationship between rarotongan an...,what is the relationship between rarotongan an...
9,0,8529165,8529165,9,35.009236,what is the relationship between rarotongan an...,what is the relationship between rarotongan an...
